# Exploration des données

## Initialisation

 ### Installation des librairies

In [ ]:
# Mettre a jour l'environnement virtuel et les requirements.txt
%pip install -r ../requirements.txt 
%pip freeze > ../requirements.txt 

### Importation des librairies

In [3]:
# Kaggle API
from kaggle.api.kaggle_api_extended import KaggleApi # mettre clé API kaggle dans C:\Users\<VotreNom>\.kaggle\kaggle.json

# Global
import os
import shutil

# Data
import pandas as pd
import numpy as np

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Images
import cv2

# Machine Learning
#import tensorflow as tf


### Téléchargement Data

In [ ]:
# Authentification
api = KaggleApi()
api.authenticate()

# URL du dataset à télécharger
dataset_url = 'tawsifurrahman/covid19-radiography-database'

# Chemin du dossier où le dataset sera extrait
destination_dir = os.path.abspath("../data")
dataset_folder = os.path.join(destination_dir, dataset_url.split('/')[-1])
print("Dossier de destination :", dataset_folder)

# Création du dossier de destination principal
os.makedirs(destination_dir, exist_ok=True)

# Vérification de l'existence du dossier du dataset
if os.path.exists(dataset_folder):
    print(f"Le dossier du dataset existe déjà : {dataset_folder}")
    print("Téléchargement ignoré.")
else:
    print("Téléchargement du dataset dans :", dataset_folder)
    api.dataset_download_files(
        dataset_url,
        path=dataset_folder,
        unzip=True
    )
    print("Dataset téléchargé avec succès dans :", dataset_folder)
    

### Analyse des Data

In [ ]:
path = os.path.join(dataset_folder,os.listdir(dataset_folder)[0])
os.listdir(path)

In [ ]:
normal_metadata = pd.read_excel(
    "../data/raw/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset/Normal.metadata.xlsx", # noqa
    sheet_name="Sheet1"
    )
normal_metadata.head()


normal_images = 

,FILE NAME,FORMAT,SIZE,URL
0,NORMAL-1,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...
1,NORMAL-2,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...
2,NORMAL-3,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...
3,NORMAL-4,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...
4,NORMAL-5,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...


In [5]:
covid_metadata = pd.read_excel(
    "../data/raw/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset/COVID.metadata.xlsx", # noqa
    sheet_name="Sheet1"
    )
covid_metadata.head()

,FILE NAME,FORMAT,SIZE,URL
0,COVID-1,PNG,256*256,https://sirm.org/category/senza-categoria/covi...
1,COVID-2,PNG,256*256,https://sirm.org/category/senza-categoria/covi...
2,COVID-3,PNG,256*256,https://sirm.org/category/senza-categoria/covi...
3,COVID-4,PNG,256*256,https://sirm.org/category/senza-categoria/covi...
4,COVID-5,PNG,256*256,https://sirm.org/category/senza-categoria/covi...


In [6]:
lung_opacity_metadata = pd.read_excel(
    "../data/raw/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset/Lung_Opacity.metadata.xlsx", # noqa
    sheet_name="Sheet1"
    )
lung_opacity_metadata.head()

,FILE NAME,FORMAT,SIZE,URL
0,Lung_Opacity-1,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...
1,Lung_Opacity-2,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...
2,Lung_Opacity-3,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...
3,Lung_Opacity-4,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...
4,Lung_Opacity-5,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...


In [7]:
pneumonia_metadata = pd.read_excel(
    "../data/raw/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset/Viral Pneumonia.metadata.xlsx", # noqa
    sheet_name="Sheet1"
    )
pneumonia_metadata.head()

,FILE NAME,FORMAT,SIZE,URL
0,Viral Pneumonia-1,PNG,256*256,https://www.kaggle.com/paultimothymooney/chest...
1,Viral Pneumonia-2,PNG,256*256,https://www.kaggle.com/paultimothymooney/chest...
2,Viral Pneumonia-3,PNG,256*256,https://www.kaggle.com/paultimothymooney/chest...
3,Viral Pneumonia-4,PNG,256*256,https://www.kaggle.com/paultimothymooney/chest...
4,Viral Pneumonia-5,PNG,256*256,https://www.kaggle.com/paultimothymooney/chest...


In [14]:
%nbqa flake8 notebooks/

Traitement des 10192 images dans le dossier : 
..\data\covid19-radiography-database\COVID-19_Radiography_Dataset\Normal\images
Traitement de l'image 10192/10192 : Normal-9999.png | Shape : (256, 256)       
Traitement terminé. 10192 images transformées en tableaux numpy.


ValueError: too many values to unpack (expected 3)

In [ ]:


fig, ax = plt.subplots(figsize=(10, 10))

plot_components(data,
                model=Isomap(n_components=2),
                images=data.reshape(-1, dsize[0], dsize[1]),
                cmap='gray',
                thumb_frac=.1)